In [1]:
import foursquare
import boto3
import time
import json
ssm = boto3.client('ssm')
ACCESS_TOKEN = ssm.get_parameters(Names=['FoursquareToken'])['Parameters'][0]['Value']
ddb = boto3.resource('dynamodb').Table('location_info')
fsq_client = foursquare.Foursquare(access_token=ACCESS_TOKEN)

def get_location():
    now = int(time.time())
    item = ddb.get_item(Key={'id': 'location'})['Item']
    if item['ts']+300 < now:
        return json.loads(item['json'])
    checkin = fsq_client.users.checkins(params={'limit': 1})['checkins']['items'][0]
    checkin_item = {
        'name': checkin['venue']['name'],
        'loc': [checkin['venue']['location']['lat'], checkin['venue']['location']['lng']],
        'ts': checkin['createdAt']
    }
    ddb.put_item(Item={'id': 'location', 'json': json.dumps(checkin_item), 'ts': now})
    return checkin_item

def lambda_handler(event, context):
    loc = get_location()
    msg = "Randall was last at {} {}".format(loc['name'], loc['ts'])
    if event['currentIntent']['name'] == 'GetLocation':
        return {"dialogAction": {
            "type": "Close",
            "fufillmentState": "Fulfilled",
            "message": {
                "contentType": "PlainText",
                "content": msg
            }
        }}

In [5]:
lambda_handler({'currentIntent': {'name': 'GetLocation'}}, None)

{'dialogAction': {'fufillmentState': 'Fulfilled',
  'message': {'content': 'Randall was last at WeWork Playa Vista 1494451741',
   'contentType': 'PlainText'},
  'type': 'Close'}}